In [1]:
# this seems right
# https://github.com/kronusme/dota2-api/blob/master/data/heroes.json
# this agrees https://liquipedia.net/dota2/MediaWiki:Dota2webapi-heroes.json
most_popular_heroes = [
    14, # pudge
    26, # lion
    74, # invoker,
    84, # ogre_magi
    41, # faceless_void
    21, # windrunner / windranger
    7, # earthshaker
    104, # legion_commander
    9, # mirana
    44, # phantom_assassin
    22, # zeus
    93, # slark
    5, # crystal_maiden
    8, # juggernaut
    86, # rubick
    35, # sniper
    6, # drow_ranger
    101, # skywrath_mage
    25, # lina
    2, # axe
    114, # monkey king
    27, # shadow shaman
    23, # kunkka
    99, # bristleback
    30, # witch_doctor
    32, # riki
    34, # tinker
    75, # silencer
    1, # anti mage
    42, # skeletonking/wraithking
    50, # dazzle
    90, # keeper of the light
    106, # ember spirit
    62, # bounty hunter
    60, # night stalker
    54, # life stealer
    17, # storm spirit
    3, # bane
    37, # warlock
    47, # viper
    113, # arc warden
    64, # jakiro
    72, # gyro
    81 # chaos knight
]

In [2]:
import pickle

with open('data/popularityDict.pkl', 'rb') as f:
    popularityDict = pickle.load(f)
    
with open('data/hero_win_percentage.pkl', 'rb') as f:
    hero_win_percentage = pickle.load(f)
    
with open('data/X_no_features_df.pkl', 'rb') as f:
    X_no_features_df = pickle.load(f)

with open('data/Y_df.pkl', 'rb') as f:
    Y_df = pickle.load(f)

In [32]:
import numpy as np

X_features = []
for x in X_no_features_df:
    X_feature = []
    for hero in x:
        win_rate = []
        for hero_id in most_popular_heroes:
            if hero_id not in x:
                try:
                    win_rate.append(hero_win_percentage[hero][hero_id])
                except:
                    win_rate.append(-1)
            else:
                win_rate.append(-1)
        X_feature.append(win_rate)
        
        popularity = [popularityDict[hero] for hero in x]
        X_feature = np.concatenate((X_feature, popularity))
    X_features.append(X_feature)

In [33]:
X_edge_features = []
#we are buiilding the edge features in the following order
edges = [[0, 1], [0, 2], [0, 3], [0, 4],
         [1, 2], [1, 3], [1, 4],
         [2, 3], [2, 4],
         [3, 4]]
for X_feature in X_features:
    edge_features = []
    for index, edge in enumerate(edges):
        node1 = edge[0]
        node2 = edge[1]
        edge_feature = np.concatenate((X_feature[node1], X_feature[node2]))
        edge_features.append(edge_feature)
    X_edge_features.append(edge_features)

In [34]:
X_train_edge_features = np.array( [(np.array(X_features[i]), np.array(edges), np.array(X_edge_features[i]) ) \
                                   for i in range(len(X_no_features_df))] )

In [35]:
Y = [[most_popular_heroes.index(i) for i in y] for y in Y_df]

In [36]:
from pystruct.learners import OneSlackSSVM
from pystruct.datasets import load_snakes
from pystruct.utils import make_grid_edges, edge_list_to_features
from pystruct.models import EdgeFeatureGraphCRF

inference = 'qpbo'
# first, train on X with directions only:
crf = EdgeFeatureGraphCRF(inference_method=inference)
ssvm = OneSlackSSVM(crf, inference_cache=50, C=.1, tol=.1, max_iter=100,
                    n_jobs=1)
ssvm.fit(X_train_edge_features[:500], np.array( Y[:500]) )

OneSlackSSVM(C=0.1, break_on_bad=False, cache_tol='auto',
             check_constraints=False, inactive_threshold=1e-05,
             inactive_window=50, inference_cache=50, logger=None, max_iter=100,
             model=EdgeFeatureGraphCRF(n_states: 44, inference_method: qpbo, n_features: 44, n_edge_features: 88),
             n_jobs=1, negativity_constraint=None, show_loss_every=0,
             switch_to=None, tol=0.1, verbose=0)

In [30]:
Y_pred = ssvm.predict(X_train_edge_features[:500])

accuracy = lambda Y_pred: np.average([sum(Y_pred[i] == Y[i])/5 for i in range(len(Y_pred))])
print("Train accuracy: ", accuracy(Y_pred))

0.21714285714285714


In [31]:
Y_pred = ssvm.predict(X_train_edge_features[70:140])
print("Test accuracy: ", accuracy(Y_pred))

0.04857142857142858


In [15]:
Y_pred[0]

array([ 4,  4,  2,  3, 23], dtype=int32)